### Import.

In [82]:
import os
from datetime import datetime
from time import sleep
import traceback
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

import spacy
from spacy import displacy

import nltk

import pyconll

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger


### Тема “Создание признакового пространства”.

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

#### Preprocessing.

In [2]:
df = pd.read_pickle('preprocessed_tweets.pickle')

df.head(3)

,id,label,tweet,tweet_source,tweet_,tweet_token,tweet_token_,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [3]:
df.shape

(49159, 10)

#### Задание 1.

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.


Загружаем предобученную модель.

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
df = pd.DataFrame({'tweet': df['tweet_token'][:2500]})

In [6]:
df.head(3)

,tweet
0,"[when, father, is, dysfunctional, and, is, so,..."
1,"[thanks, for, lyft, credit, can, not, use, cau..."
2,"[bihday, your, majesty]"


In [7]:
# %time df['ner_labels'] = df['tweet'].apply(lambda row: [word.ent_type_ for word in nlp(' '.join(row))])
# %time df['ner_indexes'] = df['ner_labels'].apply(lambda row: [ind for ind in range(len(row)) if row[ind]])
%time df['ner'] = df['tweet'].apply(lambda row: [(word.ent_type_, word.text) for word in nlp(' '.join(row)) if \
                                                 word.ent_type_])

Wall time: 34.5 s


In [8]:
df.head(3)

,tweet,ner
0,"[when, father, is, dysfunctional, and, is, so,...",[]
1,"[thanks, for, lyft, credit, can, not, use, cau...","[(ORG, wheelchair), (ORG, vans), (ORG, pdx)]"
2,"[bihday, your, majesty]","[(PERSON, bihday)]"


TOP-20 популярных NER.

In [9]:
ner_list = df['ner'].sum()
ner_count_dict = {}

for ner in ner_list:
    ner_count_dict[ner] = ner_list.count(ner)
    
ner_count_dict_sorted = sorted(list(ner_count_dict.items()), key=lambda val: val[1], reverse=True)

In [10]:
ner_count_dict_sorted[:5]

[(('DATE', 'today'), 94),
 (('DATE', 'day'), 60),
 (('DATE', 'the'), 47),
 (('DATE', 'weekend'), 37),
 (('CARDINAL', 'one'), 37)]

In [11]:
ner_count_dict_sorted = [val for val in ner_count_dict_sorted if val[0][0] in ['ORG', 'GPE', 'PERSON']]

ner_count_dict_sorted[:20]

[(('ORG', 'am'), 34),
 (('PERSON', 'bihday'), 26),
 (('GPE', 'orlando'), 18),
 (('GPE', 'america'), 12),
 (('PERSON', 'hu'), 11),
 (('PERSON', 'hea'), 9),
 (('ORG', 'house'), 9),
 (('GPE', 'london'), 8),
 (('PERSON', 'christina'), 7),
 (('GPE', 'florida'), 7),
 (('ORG', 'bong'), 7),
 (('ORG', 'bing'), 7),
 (('ORG', 'euro'), 6),
 (('GPE', 'japan'), 6),
 (('PERSON', 'hillary'), 6),
 (('GPE', 'libtard'), 6),
 (('GPE', 'sjw'), 6),
 (('GPE', 'us'), 6),
 (('ORG', 'gon'), 6),
 (('ORG', 'bull'), 6)]

Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным?

In [12]:
org_count = []
person_count = []
gpe_count = []

[org_count.append(val[1]) for val in ner_count_dict_sorted if val[0][0] == 'ORG']
[person_count.append(val[1]) for val in ner_count_dict_sorted if val[0][0] == 'PERSON']
[gpe_count.append(val[1]) for val in ner_count_dict_sorted if val[0][0] == 'GPE']

print('ORG', sum(org_count))
print('PERSON', sum(person_count))
print('PGE', sum(gpe_count))

print('\nТип PERSON самый популярный в датасете.')

ORG 613
PERSON 880
PGE 258

Тип PERSON самый популярный в датасете.


С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных.

In [13]:
print([val[0][1] for val in ner_count_dict_sorted if val[0][0] == 'ORG'][:20])
print()
print([val[0][1] for val in ner_count_dict_sorted if val[0][0] == 'PERSON'][:20])

['am', 'house', 'bong', 'bing', 'euro', 'gon', 'bull', 'positive', 'sun', 'buffalo', 'school', 'nba', 'orlando', 'usa', 'app', 'time', 'sma', 'islam', 'hill', 'social']

['bihday', 'hu', 'hea', 'christina', 'hillary', 'grimmie', 'sun', 'dance', 'dog', 'ff', 'cat', 'warcraft', 'mm', 'browser', 'tom', 'marbs', 'lil', 'hope', 'blur', 'am']


Действительно ли в топ вошли только персоны и организации или есть мусор?

В ТОП'ах в разрезе типов именованных сущностей приличная доля мусора.

#### Задание 2.

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


In [14]:
df = pd.read_pickle('preprocessed_tweets.pickle')

df = pd.DataFrame({'tweet': df['tweet_token_'][:1250]})

df.head(3)

,tweet
0,"[when, father, is, dysfunctional, and, is, so,..."
1,"[thanks, for, lyft, credit, can, not, use, cau..."
2,"[bihday, your, majesty]"


In [15]:
for val in df['tweet'].values:
    if 'trump' in val:
        print(val)
        break

['trump', 'calling', 'on', 'obama', 'to', 'resign', 'over', 'the', 'orlando', 'shootings', 'the', 'boy', 'has', 'point', 'years', 'and', 'all', 'he', 'did', 'was', 'talk', 'about', 'change']


In [16]:
df_ = pd.read_csv('train_tweets.csv')

df_.head()

for val in df_['tweet'].values:
    if 'Trump' in val:
        print('Trump')

In [17]:
df_.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


Как оказалось, NLTK не воспринимает слова с маленькой буквы как NE. Видимо это обязательное условие чтобы модуль хотя бы начал рассматривать токен в качестве кандидата на NE. Ниже эксперимент. У нас не только в предобработанном дата-сете слова переведены в нижний регистр, но и в исходных данных все слова в нижнем регистре изначально, поэтому предобработать данные с сохранением регистра символов нет возможности. Возможно, в NLTK NER есть акакой-то режим, позволяющий игнорировать это ограничение - пока такого не нашел.

In [18]:
for ind in range(1000):
    tw = [('trump' if word == 'trump' else word) for word in df['tweet'].values[ind]]
    tw = ' '.join(tw)

    for val in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(tw))):
        if hasattr(val, 'label'):
            print(val.label(), val)

In [19]:
for ind in range(1000):
    tw = [('Trump' if word == 'trump' else word) for word in df['tweet'].values[ind]]
    tw = ' '.join(tw)

    for val in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(tw))):
        if hasattr(val, 'label'):
            print(val.label(), val)

GPE (GPE Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
GPE (GPE Trump/NN)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)
PERSON (PERSON Trump/NNP)


#### Задание 3.

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.



Ну, поскольку с помощью NLTK получилось в принципе извлечь NE из корпуса, spacy отработал, конечно, получше, хотя не с идеальным качеством.

### Тема “Улучшение качества  POS-tagger'а”.

##### Baseline.

На вебинаре был рассмотрен пример тегирования с использованием корпуса на Русском языке, вам необходимо улучшить модель что бы качество классификации было выше чем с лог регрессией 

In [25]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [26]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [33]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [34]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [35]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [36]:
test_enc_labels = le.transform(test_label)

In [37]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [38]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

In [39]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [41]:
X_train.shape

(871526, 50)

In [42]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

C:\Users\Mihail\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mihail\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Mihail\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
pred = lr.predict(X_test)

In [44]:
accuracy_score(test_enc_labels, pred)

0.6331092238735551

accuracy_score логистической регрессии: 0.63310

##### Улучшенное решение.

In [78]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=100)

In [79]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [80]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.700392612812995